In [54]:
import pandas as pd
import re
import string
string.punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [55]:
lyrics = "[Verse 1]Please save a kid that needs some helpAnd I can’t begin to think of anything that could helpBut the weed is guaranteed, indeed, just what I needHow I feel, upon a time, so recent in time, made me sadWhen I recollect how it used to beLike David and Goliath, kinda like meAnd the Devil tryna rip out my soulTryna catch a nigga on sleep, no-noYou can try again and I’ll be readyWon’t let you kill me in my dream like Freddy KruegerNo, I’m not no loser, I’ll see you in Hell[Chorus]At the end of the day, dayMy mama told me don’t let no one break me, let no one break me(Yeah, yeah, yeah, no)At the end of the day, dayNobody, nobody ever could stop me, ever could stop me(Yeah, yeah, no)At the end of the day, dayYou can’t regret it if you were trying, if you were trying(Yeah, yeah, no)At the end of the day, dayI’m walking with a heart of a lion, yeah, [Verse 2]Please save a kid that needs some helpAnd I can’t see ahead of me so I move in stealthHide and seek within a dream I seem to glide above my horrorThough I feel I’ll never be complete inside the dark I borrowTo proceed and remain intact my mental is so unstableAnd they talk and judge a manThey have no clue of what I’m capable’Til I show a side of me no one had thought could be withinI told you, no I’m not a loser, I’ll see you in Hell[Chorus]At the end of the day, dayMy mama told me don’t let no one break me, let no one break me(Yeah, yeah, yeah, no)At the end of the day, dayNobody, nobody ever could stop me, ever could stop me(Yeah, yeah, no)At the end of the day, dayYou can’t regret it if you were trying, if you were trying(Yeah, yeah, no)At the end of the day, dayI’m walking with a heart of a lion, yeah[Bridge]No, no, no, no, no, no, noThe heart of a lion, the heart of a lionYeah, no, yeah (Yeah)No, no, no, no, no, no, noThe heart of a lionYeah, no, yeah (Yeah)No, no, no, no, no, no, noThe heart of a lion, the heart of a lionYeah, no, yeah (Yeah)No, no, no, no, no, no, noYeah, no, yeah (Yeah)[Verse 3]I’m just a kid who needs no helpI’ve achieved a bigger thingNo cash or check could be more blessedAnd if you had ever seen the things I’ve seen up in my slumberYou’d be shook, stay awakeNo need to run, your fate would followYou should know, not for the faint at heartThis world understands no cowardIt’s a goal, a simple code, I stay on the grind to the 25th hourAnd I know the fight in me is somewhere hiding deep withinI told you, no, I’ll never let you drag me down to Hell[Chorus]At the end of the day, dayMy mama told me don’t let no one break me, let no one break me(Yeah, yeah, yeah, no)At the end of the day, dayNobody, nobody ever could stop me, ever could stop me(Yeah, yeah, no)At the end of the day, dayYou can’t regret it if you were trying, if you were tryingAt the end of the day, day(Yeah, yeah, no)I’m walking with a heart of a lion, yeah, ‘[Outro]No, no, no, no, no, no, noYeah, no, yeah (Yeah)No, no, no, no, no, no, noYeah, no, yeah (Yeah)No, no, no, no, no, no, noYeah, no, yeah (Yeah)No, no, no, no, no, no, noYeah, no, yeah (Yeah) Yeah, yeah, yeah...’]"

In [71]:
X_final = pd.read_csv('../raw_data/rap_data.csv', dtype="string")

In [74]:
X_final.Songs.dtype

string[python]

In [75]:
def clean_text(text):

    #remove headers like [Chorus] etc
    headers = re.findall(r"\[(.*?)\]", text)
    for header in headers:
        text = text.replace(header, ' ')

    #separate lower/upper case words (like 'needHow')
    text = re.findall('[A-Z][^A-Z]*', text)
    text = " ".join(text)

    #remove punctuation
    exclude = string.punctuation + "’‘”“"
    for punctuation in exclude:
           text = text.replace(punctuation, '')

    #turn text into lowercase
    text = text.lower()

    #remove numericals
    text = ''.join(word for word in text if not word.isdigit())

    #remove stopwords
    stop_words = set(stopwords.words('english'))

    #tokenise
    word_tokens = word_tokenize(text)
    text = [w for w in word_tokens if not w in stop_words]

    #lemmatise
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in text]
    text = lemmatized

    text = ' '.join(text)

    #rejoin "wan na" to "wanna"
    wannas = re.findall(r"wan na", text)
    for wanna in wannas:
        text = text.replace(wanna, "wanna")

    return text

In [76]:
X_final['Songs'] = X_final['Songs'].apply(clean_text)

In [77]:
X_final

,Unnamed: 0,Artists,Songs,Year
0,0,ARTIST,l e,YEAR
1,1,Birdman,million,2008
2,2,Yung Joc Featuring Marques Houston & Trey Songz,time,2007
3,3,DJ Unk,step,2007
4,4,50 Cent,question,2003
...,...,...,...,...
316,316,Young Buck Featuring Latoya Williams,aint goin nowhere,2007
317,317,Ice Cube Featuring Mack 10 & Ms. Toi,put back,2002
318,318,Ice Cube Featuring Mack 10 & Ms. Toi,,2000
319,319,Guerilla Black Featuring Mario Winans,youre one,2005


In [68]:
X_final['Songs'] = X_final['Songs'].astype('str')

In [69]:
X_final['Songs'].dtype

dtype('O')

In [46]:
text_list = clean_text(lyrics).split()

In [47]:
text_list

['please',
 'save',
 'kid',
 'need',
 'help',
 'cant',
 'begin',
 'think',
 'anything',
 'could',
 'help',
 'weed',
 'guaranteed',
 'indeed',
 'need',
 'feel',
 'upon',
 'time',
 'recent',
 'time',
 'made',
 'sad',
 'recollect',
 'used',
 'like',
 'david',
 'goliath',
 'kinda',
 'like',
 'devil',
 'tryna',
 'rip',
 'soul',
 'tryna',
 'catch',
 'nigga',
 'sleep',
 'nono',
 'try',
 'ill',
 'ready',
 'wont',
 'let',
 'kill',
 'dream',
 'like',
 'freddy',
 'krueger',
 'im',
 'loser',
 'ill',
 'see',
 'hell',
 'end',
 'day',
 'day',
 'mama',
 'told',
 'dont',
 'let',
 'one',
 'break',
 'let',
 'one',
 'break',
 'yeah',
 'yeah',
 'yeah',
 'end',
 'day',
 'day',
 'nobody',
 'nobody',
 'ever',
 'could',
 'stop',
 'ever',
 'could',
 'stop',
 'yeah',
 'yeah',
 'end',
 'day',
 'day',
 'cant',
 'regret',
 'trying',
 'trying',
 'yeah',
 'yeah',
 'end',
 'day',
 'day',
 'im',
 'walking',
 'heart',
 'lion',
 'yeah',
 'please',
 'save',
 'kid',
 'need',
 'help',
 'cant',
 'see',
 'ahead',
 'move',
 's

In [60]:
data = pd.DataFrame(text_list, columns = ['lyrics'])
data

,lyrics
0,please
1,save
2,kid
3,need
4,help
...,...
321,yeah
322,yeah
323,yeah
324,yeah


In [61]:
vectorizer = TfidfVectorizer().fit(data['lyrics'])
data_vectorized = vectorizer.transform(data['lyrics'])

lda_model = LatentDirichletAllocation(n_components=2).fit(data_vectorized)

In [62]:
def print_topics(mdl, vct):
    for idx, topic in enumerate(mdl.components_):
        print("Topic %d:" % (idx))
        print([(vct.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])
        

print_topics(lda_model, vectorizer)

Topic 0:
[('yeah', 51.46826760546684), ('day', 24.468119589010666), ('heart', 9.467640799966036), ('could', 9.46764028310053), ('let', 8.467542999799804), ('lion', 8.467542294990558), ('im', 7.467418733599792), ('ever', 7.467417416648429), ('nobody', 6.467252543282194), ('break', 6.467249889504106)]
Topic 1:
[('end', 12.485717892682654), ('one', 7.485649244961411), ('trying', 6.48562275353196), ('stop', 6.48562273908155), ('need', 5.485586599455277), ('told', 5.485586524027978), ('cant', 5.4855864383811905), ('walking', 3.4854536152544457), ('within', 3.485453115382111), ('ive', 2.485316281689207)]


In [63]:
example = ["yeah"]

example_vectorized = vectorizer.transform(example)

lda_vectors = lda_model.transform(example_vectorized)

print("topic 0 :", lda_vectors[0][0])
print("topic 1 :", lda_vectors[0][1])

topic 0 : 0.7496889064131271
topic 1 : 0.25031109358687287


In [64]:
lda_vectors

array([[0.74968891, 0.25031109]])